# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
cities = pd.read_csv("../output_data/cities.csv", encoding="utf-8")
cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,sydney mines,46.2367,-60.2177,73.09,51.0,0.0,5.99,CA,1.627586e+09
1,taolanaro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bredasdorp,-34.5322,20.0403,48.27,89.0,8.0,6.76,ZA,1.627586e+09
3,mataura,-46.1927,168.8643,40.48,94.0,87.0,3.89,NZ,1.627586e+09
4,arraial do cabo,-22.9661,-42.0278,64.26,48.0,75.0,23.02,BR,1.627586e+09
...,...,...,...,...,...,...,...,...,...
606,inta,66.0317,60.1659,49.03,80.0,98.0,6.17,RU,1.627586e+09
607,songjianghe,42.1859,127.4790,67.53,97.0,100.0,3.83,CN,1.627586e+09
608,saint-joseph,-21.3667,55.6167,68.38,82.0,43.0,17.31,RE,1.627586e+09
609,cocorit,27.5833,-109.9667,95.34,35.0,21.0,10.42,MX,1.627586e+09


In [9]:
cities = cities.dropna()
cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,sydney mines,46.2367,-60.2177,73.09,51.0,0.0,5.99,CA,1.627586e+09
2,bredasdorp,-34.5322,20.0403,48.27,89.0,8.0,6.76,ZA,1.627586e+09
3,mataura,-46.1927,168.8643,40.48,94.0,87.0,3.89,NZ,1.627586e+09
4,arraial do cabo,-22.9661,-42.0278,64.26,48.0,75.0,23.02,BR,1.627586e+09
5,geraldton,-28.7667,114.6000,65.12,82.0,75.0,20.71,AU,1.627586e+09
...,...,...,...,...,...,...,...,...,...
606,inta,66.0317,60.1659,49.03,80.0,98.0,6.17,RU,1.627586e+09
607,songjianghe,42.1859,127.4790,67.53,97.0,100.0,3.83,CN,1.627586e+09
608,saint-joseph,-21.3667,55.6167,68.38,82.0,43.0,17.31,RE,1.627586e+09
609,cocorit,27.5833,-109.9667,95.34,35.0,21.0,10.42,MX,1.627586e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
humidity = cities["Humidity"]
max_humidity = humidity.max()

In [11]:
locations = cities[["Lat","Lng"]]

###  Please note the Humidity Heat Map screenshot was placed in the Images folder \Python-API-challenge\Images

In [12]:
# Plot Heatmap
fig = gmaps.figure()

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=max_humidity,point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [14]:
ideal_cities = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) 
                          & (cities["Wind Speed"] < 10) & (cities["Cloudiness"] == 0)]
ideal_cities.head()                                                                                                                

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,sydney mines,46.2367,-60.2177,73.09,51.0,0.0,5.99,CA,1.627586e+09
94,shingu,33.7333,135.9833,76.91,84.0,0.0,5.12,JP,1.627586e+09
104,bafra,41.5678,35.9069,73.18,70.0,0.0,4.18,TR,1.627586e+09
168,bay roberts,47.5999,-53.2648,73.65,54.0,0.0,8.99,CA,1.627586e+09
192,jiayuguan,39.8167,98.3000,76.23,19.0,0.0,3.83,CN,1.627586e+09


In [15]:
ideal_cities = ideal_cities.dropna()
ideal_cities.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,sydney mines,46.2367,-60.2177,73.09,51.0,0.0,5.99,CA,1.627586e+09
94,shingu,33.7333,135.9833,76.91,84.0,0.0,5.12,JP,1.627586e+09
104,bafra,41.5678,35.9069,73.18,70.0,0.0,4.18,TR,1.627586e+09
168,bay roberts,47.5999,-53.2648,73.65,54.0,0.0,8.99,CA,1.627586e+09
192,jiayuguan,39.8167,98.3000,76.23,19.0,0.0,3.83,CN,1.627586e+09


In [16]:
ideal_cities.reset_index(inplace=True)
ideal_cities.head()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sydney mines,46.2367,-60.2177,73.09,51.0,0.0,5.99,CA,1.627586e+09
1,94,shingu,33.7333,135.9833,76.91,84.0,0.0,5.12,JP,1.627586e+09
2,104,bafra,41.5678,35.9069,73.18,70.0,0.0,4.18,TR,1.627586e+09
3,168,bay roberts,47.5999,-53.2648,73.65,54.0,0.0,8.99,CA,1.627586e+09
4,192,jiayuguan,39.8167,98.3000,76.23,19.0,0.0,3.83,CN,1.627586e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
hotel_list = []

In [18]:
len(ideal_cities)

20

In [19]:
for index, row in ideal_cities.iterrows():
    lat = ideal_cities.loc[index]["Lat"]
    lng = ideal_cities.loc[index]["Lng"]
    
    # Use nearby search with a radius of 5000 for hotels
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
          "location": f'{lat},{lng}',
          "radius": 5000,
          "types": "hotel",
          "key": g_key
         }  
    
    requested = requests.get(base_url, params=params)
    hotel_jsn = requested.json()
    print(json.dumps(hotel_jsn, indent=4, sort_keys=True))
    try:
          hotel_list.append(hotel_jsn['results'][0]['name'])
    except:
          hotel_list.append("")
                
ideal_cities["Hotel Name"] = hotel_list
ideal_cities = ideal_cities.dropna()


{
    "html_attributions": [],
    "next_page_token": "Aap_uEBQ1VW2xCK-ecQvDW6uFZ4AAq9Y6iKzZiq16pZRWAW0vtvjgqc_iUIXLDlpQ6SLueZetLGfKK8OE-vD7T6jsVb9ugKc2_6x6VgbgzuIXROKlAEcs0hj3NE63jPCTA6teJCp88-_JRKclPoNBCt4JpVM0vLTljA2cJ_6r-k8bhbYJLvgcxTsIPOuUlkaZ58nBmT6pyhAuk1NMaT88SvqUMpmBkE0ptLaubX8gOdB-FGJhxMuoD8xFJ8xOnSVmQMGbdH2xhpns_SkmM0jWTm0cP1P5ir-eHODKYSvcncDImeYRILN1SHmf13Xvemw7EHQfx7U1R8yTU9MRwDiDLgzKzom__xLDE8kT_V3HphNTh7PBGXoJNMJISonk24I7iKQ2rlxQw0y5CYfm5uiDza2ZZO9i9Dv2m75aDLnKO0bKOnTKH_V6W-pLw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 46.242273,
                    "lng": -60.23061339999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 46.26370508471251,
                        "lng": -60.20520694066628
                    },
                    "southwest": {
                        "lat": 46.21911197116021,
                        "lng": -60.264997

{
    "html_attributions": [],
    "next_page_token": "Aap_uEDqs-unYifL9hIwZ1ZGQ_sBE8SZt8NgpkU68pvUR1FkemtJcO2CM24G_AdNS4wy7RGctb2qQtnnju-SlHuMfJs2eNRDRjXROqSreHsw_6x-oAgW64dOv1H3JFBjK2us8AMz3FQLmuE3eqM-oRknpwD03E1tSYCYH_ycyB7c3s73dpn2306-7rwglLW57Gx1tYWxXk-7NB_PKORDXX0zFh2cGk_ElorolnXhBZCuLXwcLhzCHreDIsOnKUyzPGVuYD8Bm5o0Fl7cXsBdAf0bQsT0Fhz_3my5wkA-L1PimUydUSNimJ7-zc9823agwQLNg9fRzLIx3dhwUU3hFvz04de282oueRXatkADlQzdxP8FqEEB1BpP-06n6nqX0gUsRXBOhq_3e7l7cuFCDGUrIwo2qXmIj-Ir7WbFka-G5bNZcr8Ihzq8-Q",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 33.7241203,
                    "lng": 135.9924819
                },
                "viewport": {
                    "northeast": {
                        "lat": 33.92213154686176,
                        "lng": 136.0122231338376
                    },
                    "southwest": {
                        "lat": 33.66953719406458,
                        "lng": 135.72237206471
 

{
    "html_attributions": [],
    "next_page_token": "Aap_uEDNpf26nWZNcqxgm5dl-LSK8IJkCLfSrjvtz73zi9PKFO_XgNSVbztXW0YiL0FiaIpHPOUBqTZBszKYT0Bff_num_GFw4PeJKCp6cIiS_y1V4i4tYoSQvyK4Gx2L1g9lEzSEmGo1SGOzJcEO5pt13VvyV2opc2U5uGgo1NDsnSdeeQCvrig3qV0awYmFuaCCHXl01adIGi0ryNDZ-18-sFu6Pi-tfv1kt9UlhdODJKSew6ClX2P1nnwFOuvzSI6ho_A6Gi1SbGWv5CQYUt23ndklOPILx9cF7cUZxeJBfgdtF_1WE2dK-ZdtxQfywJ1cRonJCLGjCnpqZsi5O-KNtOIv20xGOrmNA3GhSd-mbaz0-jv_ITEkyUD6OWk-wo7Xms_byGdzQt5GQzBdvbKjBJ7bAQWaxCZNXhNn1wBpxPoVkw0fLJn-w",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 41.561959,
                    "lng": 35.905686
                },
                "viewport": {
                    "northeast": {
                        "lat": 41.58482302323149,
                        "lng": 35.93230895823032
                    },
                    "southwest": {
                        "lat": 41.54361098003692,
                        "lng": 35.87890401661052
  

{
    "html_attributions": [],
    "next_page_token": "Aap_uEAz_r11jfjlU8CgP7rAqJ-Enn3eK-jaD0Jyvas33xSvtWA7rZPQYKBrILvAHj-OHlynb2NEufEABEhZ8AQm-uJl16nciPSIIvWiZNpgoAdwWkKmkuZu0nf264fcpD98txd0B7hvG1l0ZXswsXX5seRDlQxojOqcGgDD5E_Z4AJ8h880ElVVZQGy1MtIQ77Vjq1cUvYJaiHTEy2zduP_bY0Zq2MEhNi5-yd8V9dIYCelK8PcHlOsOh-rvxkMiCocJTtRq3FT5DPCOLx9bz5OlVlIG3YSDDmwZntEzRDwhrXeWSJzt_931g8EIjAB8UyUH0itYH3C3ylyQxtyA2Eftnb3MZ4CTWfn1BizGZywKAYz1x6FZPTmvkBKUX470MRFk-cjRlx7TaCGWf_SrT373huAKYFGtbx3F9h-lpgO7aTVfyS-jNYILA",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 47.5820518,
                    "lng": -53.2798524
                },
                "viewport": {
                    "northeast": {
                        "lat": 47.66875794604328,
                        "lng": -53.12573206069739
                    },
                    "southwest": {
                        "lat": 47.54990605891182,
                        "lng": -53.342792998735

{
    "html_attributions": [],
    "next_page_token": "Aap_uEDAXxo53bWh6FfUAhxPDEvyx3eKUkgTo27NFngTUiFqPigHwylHuOXKGlVLqnvrAw_kwSG_kD4-crouLizC2Lp7NlcR1-UKQjzcEwTb0sbLXZSeaN9s4HsWPrZ4kLujm_34Qh_hAxMjjEELDcJym-P68qZ8ILTBKV9aLiK6e7JvQhWIK9QOw-q_G-ten_bCxc7Qm2uw4EVD38ZTbJgOzmxSi3unXVUWknHxgNP2V1q6FehPQhd2Gf9_D6xqqES7hRW-8PVLeQR_zILa_ghkXwEjNCBMW8cb3CBBBelocxneMbQYd1-2QIGxKmKlNBZ-P6HE8sbGHv54KN6pyNu7V0FcrwfTw-TjxAYoR85zWi2I11xM-NxT-lZ12XNPm8-UVyK3uN4yKD8UADBaX60wwvifNtn1KF26c5z42xL9MPT4h1Ms-EvoKw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 39.772554,
                    "lng": 98.289419
                },
                "viewport": {
                    "northeast": {
                        "lat": 39.8817444,
                        "lng": 98.406601
                    },
                    "southwest": {
                        "lat": 39.6890765,
                        "lng": 98.1326294
                    }
         

...

{
    "html_attributions": [],
    "next_page_token": "Aap_uEDcCUAsCbpMxYnousutp2JhsFTmItK3AmBVEazrZmJ8r2Ap3AVkMBf9MPXtYBv72cfZvLlU_yZeC9BDlt1nJKZjXj5lS4lG5suiC-Pff7Uv-UThFMw86Dbprcr-yaiO17DSFc-bg6YChU8HrV0LLNOjZZL8bCq6Rj7IM-KejEI5TxpHtn5UsDQJp7m28hMg7EGnONqPYOAQI_J8kNqr-bbPsWOZdMbDxSEsED11AG2mHUAW6e402e7tBJLUcqCuInmIkQcGRATsd4b6wMkP8W_fwYmLgqiz5RorBUdw3wl6emOtokoot7FKclPOylUqnc8lDii5NDQnIYybEjmWuBVBGireaKKcbZtqvluIVtEUMOCgJhT9EroFHG9P2H2uQoMp43n-5jczhMBfsyjz9flIc6hvLKsju6M-pZiG2dR1Xes-aZYfCA",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 38.1977107,
                    "lng": 67.2013657
                },
                "viewport": {
                    "northeast": {
                        "lat": 38.23197775221099,
                        "lng": 67.24911689871183
                    },
                    "southwest": {
                        "lat": 38.15467201598199,
                        "lng": 67.16624740370422


{
    "html_attributions": [],
    "next_page_token": "Aap_uEDQtjG7M-GE1bzRYgb0UnJOlIRmYtcCA3T92zqZO6XcYEX1uvNxdoOyesjA06DPrMGozQq_MV0Up0-UnTeKr1JDIeQzit4NxG2OzURSKRKZVubiAwV0OUxsaJuSvSEReRcv3xzO5UO1YUV8sAkFZF11gQ4jQxOmKbiBGcAl2abp_yUCLtwRi_lKEsRsBYrPyaYG2iuTNIBcPmQp2g-IWNlavq-nUlsiVYb50Wyzr93iGrIs_xMN0MMD8AH48MaFjbXteVcK0-nAWde5ZS99CLiB4nJFAoe1msxQKz4ppzD8gAuCtlf36fVi9pf-nCMZyTB-lWt7pCzslS1b5BO7irXoQ2XUeCnz8Ce5u-P_Drpm2PJGF-jzusr0RmwJPsCIul9pv9SbHYCytYwgKwccja0D5dfU0YWdpw1M6ha5obSX98KGrbXcLg",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 41.208647,
                    "lng": 36.972055
                },
                "viewport": {
                    "northeast": {
                        "lat": 41.22104903904608,
                        "lng": 37.0044620125458
                    },
                    "southwest": {
                        "lat": 41.18983196410424,
                        "lng": 36.94897191866531
   

{
    "html_attributions": [],
    "next_page_token": "Aap_uEAjahQNLpkLZPJKq-nVlYY1wtyHyWDZ6CZ79DnLTzxv6q7IQc_aPf5rxG0UNLcivAERb4fB3EfCW8mYFh_ni7QhyQXlsDYW8gZuAihOh2VQIQJKzWiExG_eJgth-2TMOitEe5WJF4GZg4nLvgsitN740UvBuMq-3p9gWxhMFEmNA-r1kk6NxitI9c9Tw-jFV1w91CaoW6sqlCGJSOaUB4oLw3rwvuTGh0r9GvoVObek-zy9NfnduMhXjp_6UqR4STdkJA2Swh8kg3b6j6i4W2giwndse9cXezOgTZXgXK8Ul4qInvf4Kbc9Zmkju9x-JgvmA3RMdBIYsf2Rpk9bYUQ15zeMrLHGCVun_KlLZrJs2Teflm5VZsgcGsBSkb5VfFVKbUB8835DvtvpVmdsedBKfbFj5qpzgY5_51rFJVJftiovTw2yxg",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -33.5854485,
                    "lng": -70.69873609999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.52395396606174,
                        "lng": -70.6287438694405
                    },
                    "southwest": {
                        "lat": -33.75665475186723,
                        "lng": -70.828

{
    "html_attributions": [],
    "next_page_token": "Aap_uEB57DCnDQW3LiXevz0TEV21t0JdvZGRdUb6I-rf8t-QacTwsZe_L-nsEcAb_ko36grn0Zh-XdF74g-cu1Kivp7oRWurEXAAtQFG3RWDdeMx96iJveoV2x8hVjOaR1BelsXClJ8eRJP1GHP8070b1VYz81NM9auaVj1PAspGK1mifPDgA8mZGriSUSujJPwdBXuM7Iln7Oo4xfkF0GNZVPds7bF9LECyDrsvIpnEm1y9wHZdnjX5YXQ-0Sv6Oh0KjzZ1PllJ7jG6LrAo-9PPUNn1AP4iIgOI5_zRTkziwvrqwNxRnQEcbiRJ2RaMnvnk6hjYhuuQH7GW4TfJ9l0ngUQ5S7l8HZvwAQ1zroHsOhOg1DHLNorWKDtID2axt8XqbVoO1nxJHYofqBVkJWgg_pHQCiihzwtrBy2XhPsFDNf1aWfe_I9fxw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 46.42347669999999,
                    "lng": 26.4258031
                },
                "viewport": {
                    "northeast": {
                        "lat": 46.4396904935985,
                        "lng": 26.46344664520348
                    },
                    "southwest": {
                        "lat": 46.40652825894811,
                        "lng": 26.370448988

{
    "html_attributions": [],
    "next_page_token": "Aap_uEDMNkUY_OH9VLfCFIZiZbLEBdKZX_ceGMnAzYs7j-IyIm9JovPNPdxfEUR7eDEjHNJM6E6Hd_9KCHWa-dbjlS_HBEqp0nublP6FK562nX9ufyzkY9BY4gERynEqTF-k8VB6ekj0gVJ1RxORUGNBMDV5NavEdbr1LGNMrnGuREqmT1XM8Pgro04Jt30pbQxx70z55Pv0eHfyb1aKUNSHmx5EbsBTHfXkHakt7CRigwTc5JMf4msFvNPxKgNyrxooauSwo5TLvYr-GcJHysvBhHPKoX4EpQEsMcZAyLIck9zJd1Jq8nQcmZpks4dBk9sILYUCWxhvR_vmflVngMNkI_AqJhPOwu1AV3roK0pMv_kFKPb8S8VwEkz3h_Quv4eUFPCTapCDZyPRBFZUcFhIVPXcLXSIzTqerFNbG2Yx2qFAD96xzBtKTw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -7.009025599999999,
                    "lng": 23.4537973
                },
                "viewport": {
                    "northeast": {
                        "lat": -6.95247302531355,
                        "lng": 23.48789689588112
                    },
                    "southwest": {
                        "lat": -7.060985469258942,
                        "lng": 23.415941

In [20]:
ideal_cities.head()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,sydney mines,46.2367,-60.2177,73.09,51.0,0.0,5.99,CA,1.627586e+09,Sydney Mines
1,94,shingu,33.7333,135.9833,76.91,84.0,0.0,5.12,JP,1.627586e+09,Shingu
2,104,bafra,41.5678,35.9069,73.18,70.0,0.0,4.18,TR,1.627586e+09,Bafra
3,168,bay roberts,47.5999,-53.2648,73.65,54.0,0.0,8.99,CA,1.627586e+09,Bay Roberts
4,192,jiayuguan,39.8167,98.3000,76.23,19.0,0.0,3.83,CN,1.627586e+09,Jiayuguan City


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

In [22]:
hotel_info = [info_box_template.format(**row) for index, row in ideal_cities.iterrows()]


###  Please note the Hotel Map screenshot was placed in the Images folder \Python-API-challenge\Images

In [23]:
# Add marker layer ontop of heat map
# markers = gmaps.marker_layer(locations)
locations = ideal_cities[["Lat", "Lng"]]
fig = gmaps.figure(center=(31.0, 31.0), zoom_level=1.7)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)

#fig = gmaps.figure()
#markers = gmaps.marker_layer(locations)

fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))